In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"roshansalunke","key":"16ea77830d34a254c4d6bb9ed8b6d87d"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -c instacart-market-basket-analysis

 91% 179M/196M [00:01<00:00, 115MB/s]
100% 196M/196M [00:01<00:00, 133MB/s]


In [7]:
! mkdir df

In [8]:
! unzip  instacart-market-basket-analysis.zip -d df

Archive:  instacart-market-basket-analysis.zip
  inflating: df/aisles.csv.zip       
  inflating: df/departments.csv.zip  
  inflating: df/order_products__prior.csv.zip  
  inflating: df/order_products__train.csv.zip  
  inflating: df/orders.csv.zip       
  inflating: df/products.csv.zip     
  inflating: df/sample_submission.csv.zip  


In [9]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.

In [10]:
!mkdir products
!mkdir dept

In [11]:
! unzip df/products.csv.zip -d products

Archive:  df/products.csv.zip
  inflating: products/products.csv   
   creating: products/__MACOSX/
  inflating: products/__MACOSX/._products.csv  


In [12]:
! unzip df/departments.csv.zip -d dept

Archive:  df/departments.csv.zip
  inflating: dept/departments.csv    
   creating: dept/__MACOSX/
  inflating: dept/__MACOSX/._departments.csv  


In [13]:
import pandas as pd
df_product = pd.read_csv("/content/products/products.csv")
df_dept = pd.read_csv('/content/dept/departments.csv')

In [14]:
df_product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [15]:
df_dept.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [16]:
df_joined = pd.merge(df_product, df_dept, on = ['department_id'])

In [17]:
df_joined['text'] = df_joined.apply(lambda row: row['product_name']+" ->: "+row['department'],axis=1)

In [18]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_joined, test_size=.2, random_state=42)

In [19]:
train_df.head()

,product_id,product_name,aisle_id,department_id,department,text
7361,10376,Organic Caraway Seeds,104,13,pantry,Organic Caraway Seeds ->: pantry
32534,40131,Original Laundry Detergent,75,17,household,Original Laundry Detergent ->: household
1457,11913,Shelled Pistachios,117,19,snacks,Shelled Pistachios ->: snacks
5201,41392,Harvest Berry Chewy Granola Bars,3,19,snacks,Harvest Berry Chewy Granola Bars ->: snacks
38539,28627,Veganic Sprouted Red Fife Raisin Bran,121,14,breakfast,Veganic Sprouted Red Fife Raisin Bran ->: brea...


In [20]:
test_df.head()

,product_id,product_name,aisle_id,department_id,department,text
33626,24154,Free & Clear Stage 4 Overnight Diapers,56,18,babies,Free & Clear Stage 4 Overnight Diapers ->: babies
18192,27348,"Beef pot roast with roasted potatoes, carrots,...",38,1,frozen,"Beef pot roast with roasted potatoes, carrots,..."
47099,27181,Coffee Liquer,124,5,alcohol,Coffee Liquer ->: alcohol
48183,20577,Bread Rolls,43,3,bakery,Bread Rolls ->: bakery
22197,16472,French Milled Oval Almond Gourmande Soap,25,11,personal care,French Milled Oval Almond Gourmande Soap ->: p...


In [21]:
from datasets import Dataset, DatasetDict
train_dataset_dict = DatasetDict({
    'train':Dataset.from_pandas(train_df)
})

In [22]:
train_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['product_id', 'product_name', 'aisle_id', 'department_id', 'department', 'text', '__index_level_0__'],
        num_rows: 39750
    })
})

In [23]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [24]:

model_name = "ybelkada/falcon-7b-sharded-bf16"
# model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [26]:
import transformers
pipeline = transformers.pipeline(
    'text-generation',
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    trust_remote_code = True,
    device_map='auto'
)

sequences = pipeline(
   ["“Free & Clear Stage 4 Overnight Diapers” ->:","Bread Rolls ->:","French Milled Oval Almond Gourmande Soap ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:

Result: “Free & Clear Stage 4 Overnight Diapers” ->: “Free & Clear Overnight Disposable Underwear” ->
- “Free & Clear Heavy Overnight Disposable Underwear” ->
For more information about Free & Clear, please visit.
About First Quality:
First Quality is one of the largest nonwoven manufacturers in the U.S. and offers the most extensive line of baby diapers, incontinence, and other nonwovens-based products in North America. Headquartered in Asheville, North Carolina, the company serves customers across North America, with distribution facilities in North America and Latin Americ.
Result: Bread Rolls ->:
- Mix the yeast and sugar in warm water and leave for five minutes
- Put the flour and the salt into a large bowl
- Create a well in the middle and pour in the liquid
- Knead the dough for ten minutes, then leave it to proof for one hour in an oven at 37 degrees Celsius with the door open and a damp cloth over the bowl
- Knead the mixture again and shape it into a loaf
- Cover with a damp 

In [27]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [28]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120 #500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [29]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

In [30]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [31]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.938200
2,4.297200
3,4.451400
4,4.125900
5,3.957700
6,3.445700
7,2.549700
8,2.793700
9,2.373200
10,2.367200


TrainOutput(global_step=120, training_loss=2.262384596467018, metrics={'train_runtime': 578.4805, 'train_samples_per_second': 3.319, 'train_steps_per_second': 0.207, 'total_flos': 941357286835200.0, 'train_loss': 2.262384596467018, 'epoch': 0.05})

In [32]:
lst_test_data = list(test_df['text'])

In [33]:
len(lst_test_data)

9938

In [34]:
sample_size = 25
lst_test_data_short = lst_test_data[:sample_size]

In [35]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

0 Free & Clear Stage 4 Overnight Diapers ->: babies personal care household products babies -> personal care infants household products diapers -> babies disposable diapers stage -> 1 -> babies disposable diapers stage 3 -> 4 -> stage 4 -> babies disposable diapers stage 4 -> 7-18 -> babies disposable diapers size 4 -> babies disposable diapers overnight -> babies disposable diapers super pack -> babies disposable diapers size 2 -> babies
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen meat seafood meat seafood frozen beef ->: other international canned goods canned goods international international canned goods canned international canned international canned international canned beef ->: canned goods other canned international canned international canned international canned international international canned goods canned meat seafood meat seafood frozen frozen canned meat seafood frozen seafood international inte

In [36]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['babies personal care household products babies -> personal care infants household products diapers -> babies disposable diapers stage -> 1 -> babies disposable diapers stage 3 -> 4 -> stage 4 -> babies disposable diapers stage 4 -> 7-18 -> babies disposable diapers size 4 -> babies disposable diapers overnight -> babies disposable diapers super pack -> babies disposable diapers size 2 -> babies',
 'frozen meat seafood meat seafood frozen beef',
 'alcohol beverages liqueurs and specialties coffee',
 'bakery deli pasta seafood frozen seafood fish smoked haddock',
 'personal care personal care soap and body wash',
 'household supplies dry goods household',
 'deli dry goods pasta beans canned goods canned goods pasta beans hummus',
 'beverages international other missing ingredient: juice',
 'dairy eggs fish meat seafood shellfish tree nuts wheat eggs other',
 'canned goods dry goods pasta rice beans canned soups chicken',
 'dairy eggs gluten meat seafood shellfish soy other produce pant

In [37]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,babies personal care household products babies...
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,frozen meat seafood meat seafood frozen beef
2,Coffee Liquer,alcohol,alcohol beverages liqueurs and specialties coffee
3,Bread Rolls,bakery,bakery deli pasta seafood frozen seafood fish ...
4,French Milled Oval Almond Gourmande Soap,personal care,personal care personal care soap and body wash
5,Dust Pan,household,household supplies dry goods household
6,Roasted Pine Nut Hommus,deli,deli dry goods pasta beans canned goods canned...
7,Cranberry Raspberry Juice Cocktail,beverages,beverages international other missing ingredie...
8,Sweet Cream Butter Salted,dairy eggs,dairy eggs fish meat seafood shellfish tree nu...
9,Traditional Chicken Barley Soup,canned goods,canned goods dry goods pasta rice beans canned...
